# Feature Engineering
Creation of new variables to improve S&P 500 forecasting models.

In [12]:
# Imports
import pandas as pd
import numpy as np

In [13]:
# Load data
columns = ["Date", "Open", "High", "Low", "Close", "Adj Close"]
df = pd.read_csv("sp500_no_volume.csv", skiprows=3, names=columns, header=None, parse_dates=["Date"])
df = df.loc[:, ~df.columns.duplicated()]
df = df.drop(0)  # Remove extra header row
df = df.set_index("Date")
df = df.apply(pd.to_numeric, errors='coerce')

In [14]:
# Feature: Daily return
df['return'] = df['Close'].pct_change()

In [15]:
# Feature: 7-day and 21-day moving averages
df['ma7'] = df['Close'].rolling(window=7).mean()
df['ma21'] = df['Close'].rolling(window=21).mean()

In [16]:
# Feature: 7-day volatility (standard deviation)
df['volatility_7'] = df['return'].rolling(window=7).std()

In [17]:
# Feature: Future price (target for regression)
df['target'] = df['Close'].shift(-1)

In [18]:
# Feature: Upward signal (target for classification)
df['target_up'] = (df['Close'].shift(-1) > df['Close']).astype(int)

In [19]:
# Novas features derivadas
df['return_lag1'] = df['return'].shift(1)
df['return_lag2'] = df['return'].shift(2)
df['close_lag1'] = df['Close'].shift(1)
df['close_lag2'] = df['Close'].shift(2)
df['ma_diff'] = df['ma7'] - df['ma21']

# Feature: Future return (target for regression)
df['target_return'] = df['Close'].pct_change().shift(-1)

# Show the new features
df.head()

,Open,High,Low,Close,Adj Close,return,ma7,ma21,volatility_7,target,target_up,return_lag1,return_lag2,close_lag1,close_lag2,ma_diff,target_return
Date,,,,,,,,,,,,,,,,,
2015-01-07,2025.900024,2029.609985,2005.550049,2005.550049,3805480000,NaN,NaN,NaN,NaN,2030.609985,1,NaN,NaN,NaN,NaN,NaN,0.012495
2015-01-08,2062.139893,2064.080078,2030.609985,2030.609985,3934010000,0.012495,NaN,NaN,NaN,2063.449951,1,NaN,NaN,2005.550049,NaN,NaN,0.016172
2015-01-09,2044.810059,2064.429932,2038.329956,2063.449951,3364140000,0.016172,NaN,NaN,NaN,2046.130005,0,0.012495,NaN,2030.609985,2005.550049,NaN,-0.008394
2015-01-12,2028.260010,2049.300049,2022.579956,2046.130005,3456460000,-0.008394,NaN,NaN,NaN,2031.579956,0,0.016172,0.012495,2063.449951,2030.609985,NaN,-0.007111
2015-01-13,2023.030029,2056.929932,2008.250000,2031.579956,4107300000,-0.007111,NaN,NaN,NaN,2018.400024,0,-0.008394,0.016172,2046.130005,2063.449951,NaN,-0.006488


In [20]:
# Tratar valores NaN (preencher com o valor anterior e seguinte)
df = df.fillna(method='ffill').fillna(method='bfill')

/tmp/ipykernel_1683/1753909209.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')


In [21]:
df.head()

,Open,High,Low,Close,Adj Close,return,ma7,ma21,volatility_7,target,target_up,return_lag1,return_lag2,close_lag1,close_lag2,ma_diff,target_return
Date,,,,,,,,,,,,,,,,,
2015-01-07,2025.900024,2029.609985,2005.550049,2005.550049,3805480000,0.012495,2029.924282,2028.764753,0.010758,2030.609985,1,0.012495,0.012495,2005.550049,2005.550049,-5.074759,0.012495
2015-01-08,2062.139893,2064.080078,2030.609985,2030.609985,3934010000,0.012495,2029.924282,2028.764753,0.010758,2063.449951,1,0.012495,0.012495,2005.550049,2005.550049,-5.074759,0.016172
2015-01-09,2044.810059,2064.429932,2038.329956,2063.449951,3364140000,0.016172,2029.924282,2028.764753,0.010758,2046.130005,0,0.012495,0.012495,2030.609985,2005.550049,-5.074759,-0.008394
2015-01-12,2028.260010,2049.300049,2022.579956,2046.130005,3456460000,-0.008394,2029.924282,2028.764753,0.010758,2031.579956,0,0.016172,0.012495,2063.449951,2030.609985,-5.074759,-0.007111
2015-01-13,2023.030029,2056.929932,2008.250000,2031.579956,4107300000,-0.007111,2029.924282,2028.764753,0.010758,2018.400024,0,-0.008394,0.016172,2046.130005,2063.449951,-5.074759,-0.006488


In [22]:
df.to_csv('features.csv')